In [13]:
import numpy as np
import gym
import torch
from torch import nn
import torch.nn.functional as F
import random
from collections import deque
import copy

from tqdm.std import tqdm
from torch.distributions.normal import Normal


env = gym.make('Pendulum-v0')
n_state = int(np.prod(env.observation_space.shape))
n_action = int(np.prod(env.action_space.shape))
print("# of state", n_state)
print("# of action", n_action)

# SEED = 1234
# torch.manual_seed(SEED)
# np.random.seed(SEED)
# random.seed(SEED)
# env.seed(SEED)

# of state 3
# of action 1


 Given certain policy, how can we compute the value function for each state.

In [14]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
IF_TARGET = True
IF_REPLAY = True
IF_PER = True


def run_episode(env, policy, render=False):
    """ Runs an episode and return the total reward """
    obs = env.reset()
    states = []
    rewards = []
    actions = []
    while True:
        if render:
            env.render()

        states.append(obs)
        action = policy(obs)
        actions.append(action)
        obs, reward, done, _ = env.step(action)
        rewards.append(reward)
        if done:
            break

    return states, actions, rewards



In [15]:
class Policy():
    def __init__(self, n_state, n_action):
        self.q_net1 = nn.Sequential(
            nn.Linear(n_state + n_action, 400),
            nn.ReLU(),
            nn.Linear(400, 300),
            nn.ReLU(),
            nn.Linear(300, 1)
        )
        self.q_net2 = nn.Sequential(
            nn.Linear(n_state + n_action, 400),
            nn.ReLU(),
            nn.Linear(400, 300),
            nn.ReLU(),
            nn.Linear(300, 1)
        )
        self.actNet = nn.Sequential(
            nn.Linear(n_state, 400),
            nn.ReLU(),
            nn.Linear(400, 300),
            nn.ReLU(),
            nn.Linear(300, n_action * 2),
        )
        self.q_net1_target = copy.deepcopy(self.q_net1)
        self.q_net2_target = copy.deepcopy(self.q_net2)
        # self.noise = 2
        self.noise = 1.75
        self.act_lim = 2
        self.alpha = 0.5
        self.gamma = 0.95
        self.ro = 0.9

        self.q_net1_optimizer = torch.optim.Adam(self.q_net1.parameters(), lr=1e-3)
        self.q_net2_optimizer = torch.optim.Adam(self.q_net2.parameters(), lr=1e-3)
        self.actNet_optimizer = torch.optim.Adam(self.actNet.parameters(), lr=1e-3)

        self.replaybuff = ReplayBuffer(50000)

        self.q_net1.to(device)
        self.q_net1_target.to(device)
        self.q_net2.to(device)
        self.q_net2_target.to(device)
        self.actNet.to(device)

        

    def update(self):
        
        obs, act, reward, next_obs, done = self.replaybuff.sample(64)
        output = self.actNet(next_obs)
        mu_ = self.act_lim * torch.tanh(output[:, :n_action])
        var_ = torch.abs(output[:, n_action:]).clamp(1e-3, 2)
        dist_ = Normal(mu_, var_)
        act_ = dist_.sample()
        logprob_ = dist_.log_prob(act_).squeeze()
        with torch.no_grad():
            q_input = torch.cat([next_obs, act_], axis=1)
            y1 = reward + self.gamma * (1 - done) * self.q_net1_target(q_input).squeeze()
            y2 = reward + self.gamma * (1 - done) * self.q_net2_target(q_input).squeeze()
            y = torch.min(y1, y2) - self.alpha*logprob_
        
        q_input = torch.cat([obs, act], axis=1)

        self.q_net1_optimizer.zero_grad()
        Q_net1 = self.q_net1(q_input).squeeze()
        q_net1_loss = F.mse_loss(Q_net1, y)
        q_net1_loss.backward()
        self.q_net1_optimizer.step()

        self.q_net2_optimizer.zero_grad()
        Q_net2 = self.q_net2(q_input).squeeze()
        q_net2_loss = F.mse_loss(Q_net2, y)
        q_net2_loss.backward()
        self.q_net2_optimizer.step()
        
        self.actNet_optimizer.zero_grad()
        output = self.actNet(obs)
        mu = self.act_lim * torch.tanh(output[:, :n_action])
        var = torch.abs(output[:, n_action:]).clamp(1e-3, 2)
        dist = Normal(mu, var)
        act = dist.rsample()
        q_input = torch.cat([obs, act], axis=1)
        min_q = torch.min(self.q_net1(q_input), self.q_net2(q_input))
        actNet_loss = (self.alpha * logprob_ - min_q).mean()
        actNet_loss.backward()
        self.actNet_optimizer.step()

        self._copy_nets()
        return q_net1_loss.item(), q_net2_loss.item(), actNet_loss.item()
        
    def _copy_nets(self):
        for target_param, real_param in zip(self.q_net1_target.parameters(), self.q_net1.parameters()):
            target_param.data.copy_(self.ro * target_param.data + ((1 - self.ro) * real_param.data))
        for target_param, real_param in zip(self.q_net2_target.parameters(), self.q_net2.parameters()):
            target_param.data.copy_(self.ro * target_param.data + ((1 - self.ro) * real_param.data))

    def __call__(self, state):
        with torch.no_grad():
            state = torch.FloatTensor(state).to(device)
            # calculate old logprob
            output = self.actNet(state)
            mu = self.act_lim*torch.tanh(output[:n_action])
            var = torch.abs(output[n_action:])
            dist = Normal(mu, var)
            action = dist.sample()
            action = action.detach().cpu().numpy()
        return np.clip(action, -self.act_lim, self.act_lim)



In [16]:


class ReplayBuffer:
    def __init__(self, size):
        self.buff = deque(maxlen=size)

    def add(self, obs, act, reward, next_obs, done):
        self.buff.append([obs, act, reward, next_obs, done])

    def sample(self, sample_size):
        if(len(self.buff) < sample_size):
            sample_size = len(self.buff)

        sample = random.sample(self.buff, sample_size)
        obs = torch.FloatTensor([exp[0] for exp in sample]).to(device)
        act = torch.FloatTensor([exp[1] for exp in sample]).to(device)
        reward = torch.FloatTensor([exp[2] for exp in sample]).to(device)
        next_obs = torch.FloatTensor([exp[3] for exp in sample]).to(device)
        done = torch.FloatTensor([exp[4] for exp in sample]).to(device)
        return obs, act, reward, next_obs, done

    def __len__(self):
        return len(self.buff)


In [17]:
loss_q_net1_list,loss_q_net2_list , loss_actNet_list, reward_list = [], [], [], []
update_index = 0
loss_q_net1, loss_q_net2, actNetloss = 0, 0, 0

policy = Policy(n_state, n_action)
for i in tqdm(range(500)):
    obs, rew = env.reset(), 0
    
    while True:
        act = policy(obs)
        next_obs, reward, done, _ = env.step(act)
        rew += reward

        update_index += 1
        if len(policy.replaybuff) > 2e3 and update_index > 4:
            update_index = 0
            loss_q_net1, loss_q_net2, actNetloss = policy.update()

        policy.replaybuff.add(obs, act, reward, next_obs, done)
        policy.replaybuff.add(obs, act, reward, next_obs, done)
        obs = next_obs
        
        if done:
            break
    if i > 0 and i % 50 == 0:
        print("itr:({:>5d}) loss:{:>3.4f} actloss:{:>3.4f} reward:{:>3.1f}".format(
            i, np.mean(loss_q_net1_list[-50:]), np.mean(loss_q_net2_list[-50:]), np.mean(loss_actNet_list[-50:]), np.mean(reward_list[-50:])))

    if policy.noise > 0.005:
        policy.noise -= (1/200)
    
    loss_q_net1_list.append(loss_q_net1), loss_q_net2_list.append(loss_q_net2), reward_list.append(rew), loss_actNet_list.append(actNetloss)



 10%|█         | 51/500 [00:20<02:54,  2.58it/s]

itr:(   50) loss:70.2082 actloss:70.0354 reward:111.9


 20%|██        | 101/500 [00:39<02:34,  2.58it/s]

itr:(  100) loss:48.9310 actloss:48.8596 reward:127.5


 30%|███       | 151/500 [00:59<02:14,  2.59it/s]

...

 80%|████████  | 401/500 [02:39<00:39,  2.49it/s]

itr:(  400) loss:45.2263 actloss:45.7462 reward:94.7


 90%|█████████ | 451/500 [03:01<00:21,  2.32it/s]

itr:(  450) loss:54.6006 actloss:54.7363 reward:90.4


100%|██████████| 500/500 [03:22<00:00,  2.47it/s]


In [18]:
scores = [sum(run_episode(env, policy, False)[2]) for _ in range(100)]
print("Final score:", np.mean(scores))

import pandas as pd
df = pd.DataFrame({'loss_q_net1': loss_q_net1_list, 'loss_q_net2': loss_q_net2_list, 'reward': reward_list, 'act loss': loss_actNet_list})
df.to_csv("./SAC.csv",
          index=False, header=True)


Final score: -490.466607222689


Based on expiramenting with alpha I believe that it is important to have alpha be around 0.2. If alpha is 0 or 0.5 it is either too influenced or not influenced enough by the bonus reward at each time step proportional to the entropy of the policy at that timestep. Alpha needs to be set just right to account for the tradeoff between rewarding expected return and entropy, a measure of randomness in the policy.
higher alpha corresponding to more exploration, and lower alpha corresponding to more exploitation

I think that the most significant contribution of SAC is the soft actor critic q learning technique. I believe that the twin qnets are simply a trick to make training networks, and updting the policy easier.

alpha = 0.0
![reward alpha 0.0](./Data/SAC_alpha00.png)

##############################################################
################################################
###########################################################

alpha = 0.2
![reward alpha 0.2](./Data/SAC_alpha02.png)

#############################################################
################################################
###########################################################

alpha = 0.5
![reward alpha 0.2](./Data/SAC_alpha05.png)